In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
from citipy import citipy
from config import weather_api_key
from datetime import datetime

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Import the API key.
from config import weather_api_key

In [3]:
# STEP 3 Create a new set of 2,000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [4]:
# Add the latitudes/longitudes and cities lists
coordinates = list(lat_lngs)
cities = []

# STEP 4 Get the nearest city using the `citipy` module
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)

In [5]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("* = complete, 's' = skipped\nBeginning Data Retrieval..\n[", end = '')
#print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    print("*" , end = '')
    # Add 1 to the record count.
    record_count += 1
    # STEP 5 Perform an API call with the OpenWeatherMap
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # STEP 6 Retrieve the following information from the API call
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
              
        # GET RAINFALL & SNOWFALL - DIFFERENT API CALL - NEEDS TRY EXCEPT BLOCKS
        try:
            rain_inches = round(city_weather["rain"]["1h"]/25.4,1)
        except:
            rain_inches = round(0,1)
        try:    
            snow_inches = round(city_weather["snow"]["1h"]/25.4,1)
        except:    
            snow_inches = round(0,1)
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current Description": city_description,
                           })

# If an error is experienced, skip the city.
    except:
        #print("City not found. Skipping...")
        print("s", end = '')
        pass

# Indicate that Data Loading is complete.
#print("-----------------------------")
print("]\n...Data Retrieval Complete", end='')
#print("-----------------------------")

* = complete, 's' = skipped
Beginning Data Retrieval..
[**s***********************************************s******************s*********s**s****s***********************************************s****s***********************s********************************s**************s**s****s**s*******************************s*******s**************s*******s*****s************************s**s*****s********s*********************************s***s****************************************s*************s*******s******s*****************************************************s****s*******s**************************s***********s******s*****************************s***************************s*s******************************************************s********s**********s********************s************************s*****************************************************************s*******************]
...Data Retrieval Complete

In [6]:
# STEP 7 Add the data to a new DataFrame
city_data_df = pd.DataFrame(city_data)

# Reorder to match challenge prompt
new_column_order = ["City","Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Chapais,CA,49.7834,-74.8492,37.18,93,95,5.30,overcast clouds
1,Saskylakh,RU,71.9167,114.0833,33.08,94,100,9.82,overcast clouds
2,Rikitea,PF,-23.1203,-134.9692,75.97,76,44,16.46,scattered clouds
3,Cidreira,BR,-30.1811,-50.2056,58.98,78,38,7.16,scattered clouds
4,Khatanga,RU,71.9667,102.5000,33.44,78,100,8.52,overcast clouds
5,Mataura,NZ,-46.1927,168.8643,49.26,73,78,9.28,broken clouds
6,Tilichiki,RU,60.4667,166.1000,46.90,61,1,3.24,clear sky
7,Copala,MX,16.6500,-98.9833,80.47,71,30,3.65,scattered clouds
8,Tank,PK,32.2217,70.3793,83.71,13,0,7.31,clear sky
9,Sur,OM,22.5667,59.5289,93.07,36,4,17.67,clear sky


In [7]:
# STEP 8 Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder
output_data_file = "WeatherPy_database.csv"
city_data_df.to_csv(output_data_file)